# Structural optimization and Schur's complement

After a geometric and a graph-theoretic application we next study an engineering
task. This allows us to learn about a handy lemma that draws a connection
between semidefinite programming and the matrix inverse: Schur's complement.

## Preamble

In [ ]:
import numpy as np
import picos as pc
from itertools import chain, product, combinations
from plotly import offline, graph_objects as go

LAYOUT = dict(
    scene=dict(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False),
    yaxis=dict(scaleanchor="x1"), margin=dict(l=10, r=10, t=10, b=10),
    paper_bgcolor="white", plot_bgcolor="rgba(0,0,0,0)",
)

offline.init_notebook_mode()


## Problem 3: Truss topology design

### Setting

A truss is a structure made of bars and joints that is designed to withstand an
external force, typically from a weight being carried. Classical examples are
truss bridges, cantilevers, and cranes. In a truss design problem, one has a
certain amount of material to spend on the construction  with the goal to
minimize the *compliance* of the truss,

$$
    \frac{1}{2} f^T d,
$$

where $f$ is a given vector of external forces that act on to the trusses joints
(the *load*) and where $d$ is the *displacement* of the joints that results from
the bars being compressed or tensioned in force equilibrium (it is assumed that
the bars cannot bend).

Given the displacements of the joints, the forces that cause them can be derived
using the *stiffness matrix* $K$ of the structure:

$$
    f = K d.
$$

This matrix can be decomposed as

$$
    K = K(x) = \sum_{i=1}^m x_i v_i v_i^T
$$

where $x_i$ are the cross-sectional areas of the $m$ bars and where the $v_i$
depend only on the length, orientation and Young's modulus of elasticity of bar
$i$.

---

<div class="alert alert-info">

   **Task 3.1:** Argue that $K(x)$ is positive semidefinite for $x \geq 0$.

</div>

---

Under normal conditions (when the truss is properly fixed to its surroundings,
see, e.g., Aranda and Bellido ([2016](#references))), $K(x)$ is further positive
definite. Then, it is invertible, and we can derive the joint displacements
under force from its inverse, which is called the *flexibility matrix*:

$$
    d = K^{-1} f.
$$

Thus, the compliance for a given vector of bar cross-sections $x$ is

$$
    \frac{1}{2} f^T {K(x)}^{-1} f.
$$

We will later minimize this quantity using semidefinite programming.

### Preparations

In the following we build a playground that allows us to define 2D truss
structures and analyze how they react to a load for different choices of $x$.
For this we'll need some constants:

In [ ]:
# Defaults are estimates for steel on earth.
g = 9.81  # gravity in m/s²
D = 7900  # bar density in kg/m³
E = 200   # Young elasticity modulus in GPa

The `Truss` class allows us to plot truss constructions normally and under load:

In [ ]:
class Truss:
    def __init__(self, nodes, dof, bars):
        # An n×d matrix whose rows are the x and y coordinates of n nodes.
        self.nodes = nodes

        # An n×d binary matrix whose rows are the degrees of freedom for each
        # node, i.e., dof[i, j] denotes whether node i can move along axis j.
        self.dof = dof

        # An m×2 matrix whose rows are node index pairs and represent possible
        # bars that can be installed between those two nodes.
        self.bars = bars

    @property
    def n(self):
        """Number of nodes."""
        return self.nodes.shape[0]

    @property
    def d(self):
        """Dimensionality."""
        return self.nodes.shape[1]

    @property
    def m(self):
        """Number of bars."""
        return self.bars.shape[0]

    @property
    def mask(self):
        """A mask to select/project onto degrees of freedom."""
        return np.bool_(self.dof.ravel())

    def stiffness_matrix(self, x):
        """Stiffness matrix K for given cross-sectional areas of the bars."""
        if len(x) != self.m:
            raise ValueError(f"x should be a container of size {self.m}")

        n, d, m = self.n, self.d, self.m

        # Vectors v_i such that K(x) = sum_i x_i v_i v_i^T.
        # NOTE: Vector length depends on the degrees of freedom of the truss.
        vectors = []
        for i, j in self.bars:
            u = self.nodes[i]
            w = self.nodes[j]
            l = np.linalg.norm(w - u)  # bar length
            a = (w - u) / l  # direction from joint u to joint w
            v = np.zeros(d*n)
            v[d*i:d*(i+1)] = a
            v[d*j:d*(j+1)] = -a
            v = v[self.mask]  # limit to degrees of freedom
            v *= (E * 1e9 / l)**0.5  # 1e9 as E is in GPa
            vectors.append(v)

        # Matrices K_i such that K = sum_i x_i K_i.
        matrices = [np.outer(v, v) for v in vectors]

        # Use pc.sum for better strings when x is a PICOS type.
        return pc.sum([x_k*A_k for x_k, A_k in zip(x, matrices)])

    def displacement(self, x, f):
        """Compute node displacements under force f for cross-sections x."""
        # Load the stiffness matrix.
        K = self.stiffness_matrix(x)

        # Vectorize the force and project it onto the degrees of freedom.
        f = f.ravel()[self.mask]

        # Compute the node displacements using least-squares regression:
        # Choose d minimizing ‖Ad - f‖².
        d = np.zeros(self.n*self.d)
        d[self.mask] = np.linalg.lstsq(K, f, rcond=None)[0]
        return d.reshape(self.n, self.d)

    def print_stats(self, x=None, f=None):
        """Show information about the truss."""
        maxs = np.max(self.nodes, axis=0)
        mins = np.min(self.nodes, axis=0)
        size_str = " x ".join((f"{s:.1f}m" for s in maxs - mins))

        if True:
            print(
                f"Max. truss size:    {size_str}\n"
                f"Nodes:              {self.n}\n"
                f"Bar locations:      {self.m}"
            )
        if x is not None:
            lengths = np.array([
                np.linalg.norm(self.nodes[i] - self.nodes[j])
                for i, j in self.bars
            ])
            volume = lengths.dot(x)
            mass = volume*D
            print(
                f"\n"
                f"Max. cross-section: {np.max(x):.1e} m²\n"
                f"as tubular diam.:   {(np.max(x)/np.pi)**0.5 * 200:7.1f} cm\n"
                f"Total volume:       {volume:.1e} m³\n"
                f"Total mass:         {mass:7.1f} kg"
            )
        if x is not None and f is not None:
            displacement = self.displacement(x, f)
            max_displacement = np.max(np.linalg.norm(displacement, axis=1))
            force_sum = np.sum(np.linalg.norm(f, axis=1))
            print(
                f"\n"
                f"Sum of forces:      {force_sum:.1e} N\n"
                f"in mass pulling:    {force_sum / g:7.0f} kg\n"
                f"Max. displacement:  {max_displacement:.1e} m"
            )

    def plot(self, x, f=None, line_width_factor=2000, force_length=40):
        """Draw the truss and its displacement under f."""
        def draw(nodes, name, color="black", group=1):
            legend = int(np.where(x == np.max(x))[0][0])
            min_x = 1e-4 / line_width_factor
            return [
                go.Scatter(
                    name=name, mode="lines", x=nodes[[i,j],0], y=nodes[[i,j],1],
                    line_color=color, showlegend=k == legend, legendgroup=group,
                    line_width=(x[k]*line_width_factor)**0.5
                )
                for k, (i, j) in enumerate(self.bars) if x[k] > min_x
            ]

        traces = draw(self.nodes, "normal")
        annotations = []

        if f is not None:
            displaced_nodes = self.nodes + self.displacement(x, f)
            traces += draw(displaced_nodes, "displaced", color="red", group=2)

            # Draw forces.
            forced_nodes = np.where((f != 0).any(axis=1))[0]
            for k, i in enumerate(forced_nodes):
                v = f[i]
                mag = np.linalg.norm(v)
                x, y = displaced_nodes[i]
                dx, dy = v / mag * force_length

                annotations.append(dict(
                    text=f"{mag:.1e} N ({mag / g / 1e3:.1f} t)", x=x, y=y,
                    ax=dx, ay=-dy, font_color="orange", arrowcolor="orange"
                ))

        # Draw fixtures.
        for pattern, name, symbol in zip(
            ((0, 0), (0, 1), (1, 0)),
            ("fixed", "v-fixed", "h-fixed"),
            ("circle", "diamond-tall", "diamond-wide")
        ):
            fixed_nodes = np.where((self.dof == pattern).all(axis=1))[0]
            x, y = self.nodes[fixed_nodes].T
            traces.append(go.Scatter(
                name=name, mode="markers", x=x, y=y,
                marker=dict(color="black", symbol=symbol)
            ))

        go.Figure(traces, LAYOUT | dict(annotations=annotations)).show()

Let's try it out on a small truss:

In [ ]:
# Construct a small truss. It's leftmost nodes are fixed to the wall.
small_truss = Truss(
    nodes=np.array([[0, 1], [1, 1], [0, 0], [1, 0], [2, 0]]),
    dof=np.array([[0, 0], [1, 1], [0, 0], [1, 1], [1, 1]]),
    bars=np.array([[0, 1], [0, 2], [1, 2], [1, 3], [1, 4], [2, 3], [3, 4]]),
)

# Give all bars the cross-section area of a 20 mm diameter tube.
diam = 2e-2  # in m
area = np.pi * (diam / 2) ** 2  # in m^2
x = np.full(small_truss.m, area)

# Define a force matrix: 20t pulling on the outer right tip.
mass = 2e4  # in kg
F = np.array([[0, 0], [0, 0], [0, 0], [0, 0], [0, -mass * g]])

# Plot the truss (with unit bar cross-sections) under stress.
small_truss.plot(x, F, line_width_factor=4000)
small_truss.print_stats(x, F)

Note that this is only physically accurate under simplifying assumptions. In
particular, the bars are not allowed to bend (the displacement comes from
compression and tension only) and neither the bars nor the joints can fail and
break. You'll also need very good wall plugs if you really want to carry 20 tons
using a 20 kg steel cantilever!

Now for a slightly bigger construction:

In [ ]:
def crane_structure(
    *, width, offset, pillar_width, foot_width, height, top_height,
    max_bar_length
):
    pw, fw, th = pillar_width, foot_width, top_height

    nodes = np.array([
        (x, y) for x, y in product(range(width + 1), range(height + 1))
        if y >= height - top_height
        or x in range(offset, offset + pw + 1)
        or (y == 0 and x in range(offset - fw, offset + pw + fw + 1))
    ])
    n = len(nodes)

    dof = np.ones(nodes.shape)
    dof[nodes[:, 1] == 0] = 0

    def redundant(i, j):
        d = np.abs(nodes[i] - nodes[j])
        if 0 in d and not 1 in d:
            return True
        elif d[0] == d[1] and d[0] != 1:
            return True
        else:
            return False

    bars = np.array([
        (i, j) for i, j in combinations(range(n), 2)
        if not redundant(i, j)
        and np.linalg.norm(nodes[i] - nodes[j]) <= max_bar_length
    ])

    tip_up = np.zeros((n, 2))
    tip_up[n - top_height - 1, 1] = 1

    tip_rt = np.zeros((n, 2))
    tip_rt[n - top_height - 1, 0] = 1

    return Truss(nodes, dof, bars), tip_up, tip_rt

The size we can work with depends on whether MOSEK is available, as CVXOPT
struggles with larger structures.

In [ ]:
# Enable if you have MOSEK and a really fast PC. :)
give_me_an_even_larger_crane = False

# Define a crane structure and a maximum crane weight to obey.
if give_me_an_even_larger_crane:
    crane, tip_up, tip_rt = crane_structure(
        width=12, offset=2, pillar_width=2, foot_width=2, height=14,
        top_height=2, max_bar_length=5
    )
    W = 2500
elif "mosek" in pc.available_solvers():
    crane, tip_up, tip_rt = crane_structure(
        width=9, offset=2, pillar_width=1, foot_width=2, height=10,
        top_height=2, max_bar_length=4,
    )
    W = 2000
else:
    crane, tip_up, tip_rt = crane_structure(
        width=5, offset=1, pillar_width=1, foot_width=1, height=6,
        top_height=1, max_bar_length=1.5
    )
    W = 200

# Derive the maximum volume V from the maximum weight W.
V = W / D

# Determine the lengths of all bars.
l = np.array([
    np.linalg.norm(crane.nodes[i] - crane.nodes[j]) for i, j in crane.bars
])
L = np.sum(l)

# Define a force to withstand.
F = tip_up * -1e4 * g  # carry 10 tons at the tip
# F += tip_rt * 1e4 * g  # pull also to the side

# Test a uniform design (every bar has same cross-section) with maximum weight.
x = np.full(crane.m, V / L)
crane.plot(x, F)
crane.print_stats(x, F)

### The problem

Recall that we want to choose the bar cross-sections $x$ to minimize the
compliance $f^T {K(x)}^{-1} f$. Of course, we cannot just make every bar as
heavy as we like, for there is a bound $W$ on the total weight of the crane
which translates to a maximum volume $V$ of material that we may spend (defined
above as `V`). The volume of bar $i$ is simply $x_i l_i$, where $l_i$ is its
length (see the definition of `l` above). Thus, the problem we'd like to solve
is

$$
\begin{align*}
    \text{minimize} ~&~ \frac{1}{2} f^T {K(x)}^{-1} f \tag{1} \\
    \text{subject to}
    ~&~ l^T x \leq V, \\
    ~&~ x \geq 0, \\
    \text{where}
    ~&~ x \in \mathbb{R}^m.\\
\end{align*}
$$

You may find that the difficult part about this formulation is the matrix
inverse: without it the problem would just be an LP that can be solved easily.
We will next learn a technique that allows us to get rid of this inverse: the
Schur complement.

**Lemma (Schur complement):** Let
$$
    P \in \mathbb{S}^p,
    \qquad
    Q \in \mathbb{R}^{r \times p},
    \qquad
    R \in \mathbb{S}^r,
    \quad
    \text{and}
    \quad
    A = \begin{bmatrix}
        P & Q^T \\
        Q & R
    \end{bmatrix}
$$
for some $r, p \in \mathbb{Z}_{\geq 1}$. Then, the following implications are
true:
$$
    \begin{align}
        R \succ 0 \quad\Longrightarrow\quad (
            A \succeq 0
            &\quad\Longleftrightarrow\quad
            P - Q^T R^{-1} Q \succeq 0
        ), \tag{Schur1} \\
        P \succ 0 \quad\Longrightarrow\quad (
            A \succeq 0
            &\quad\Longleftrightarrow\quad
            R - Q P^{-1} Q^T \succeq 0
        ). \tag{Schur2}
    \end{align}
$$

---

<div class="alert alert-info">

   **Bonus task 3.2:** Prove (Schur1) by filling in the gaps:

   <details>
   <summary style='display: list-item'>Detailed instructions</summary>

   - (a) Use a definition of $A \succeq 0$.
   - (b) Partition $z$ into $u$ and $v$ and insert the definition of $A$.
     Rewrite the scalar $u^T Q^T v$ as its transpose $v^T Q u$ to simplify a
     bit.
   - (c) Copy and paste, then think about why this makes sense. :-)
   - (d) Use the following: For $u$ and $R \succ 0$ fixed, the convex quadratic
     problem $\text{minimize}~v^T R v + 2 v^T Q u$ has the unique optimum
     solution $v^* = -R^{-1} Q u$.
   - (e) Simplify.
   - (f) Confirm that the definition of positive semidefiniteness applies.

   </details>
</div>

<div hidden>

$$
\providecommand{\TODO}{}\renewcommand{\TODO}{{\color{red}{[\ldots]}}}
$$

</div>

$$
\begin{align*}
    & & A &\succeq 0 \\
    &\overset{\text{(a)}}\Longleftrightarrow &
        \forall z \in \mathbb{R}^{p + r} \colon
        \TODO &\geq 0 \\
    &\overset{\text{(b)}}\Longleftrightarrow &
        \forall (u, v) \in \mathbb{R}^p \times \mathbb{R}^r \colon
        \TODO &\geq 0 \\
    &\overset{\text{(c)}}\Longleftrightarrow & 
        \forall u \in \mathbb{R}^p \colon
        \inf_{v \in \mathbb{R}^r} \left( \TODO \right) &\geq 0 \\
    &\overset{\text{(d)}}\Longleftrightarrow &
        \forall u \in \mathbb{R}^p \colon
        \TODO &\geq 0 \\
    &\overset{\text{(e)}}\Longleftrightarrow &
        \forall u \in \mathbb{R}^p \colon
        u^T \left( \TODO \right) u &\geq 0 \\
    &\overset{\text{(f)}}\Longleftrightarrow &
        P - Q^T R^{-1} Q &\succeq 0.
\end{align*}
$$

---

<div class="alert alert-info">

   **Task 3.3:** Use a Schur complement to rewrite problem (1) as an SDP.

</div>

<div hidden>

$$
\providecommand{\TODO}{}\renewcommand{\TODO}{{\color{red}{[\ldots]}}}
$$

</div>

$$
\begin{align*}
    \text{minimize} ~&~ \TODO \tag{2} \\
    \text{subject to}
    ~&~ \TODO, \\
    ~&~ l^T x \leq V, \\
    ~&~ x \geq 0, \\
    \text{where}
    ~&~ x \in \mathbb{R}^m.\\
\end{align*}
$$


---

<div class="alert alert-info">

   **Task 3.4:** Complete the code below to solve problem (2).

   <details>
   <summary style='display: list-item'>Recommended hints</summary>

   - You can obtain $K(x)$ as `crane.stiffness_matrix(x)` where `x` is a PICOS
     variable of suited size.
   - To improve the numeric stability, scale $f$ and $K$ to be of similar
     magnitude as an estimate of $x$ at optimality.$^1$
   - The force $f$ is given as an $n \times d$ matrix `F` but we'll need it as a
     vector, with entries corresponding to the fixed joints removed:
     `F.ravel()[crane.mask]`.

   </details>

   <details>
   <summary style='display: list-item'>Hint footnotes</summary>

   1. We may scale the objective function $\frac{1}{2} f^T {K(x)}^{-1} f$ of
     problem (1) as we like: mathematically, this has no effect on the sets of
     feasible and optimal solutions. Numerically, however, this makes a
     difference: Since both $f$ and $K(x)$ are by magnitudes larger than an
     optimal $x$ (i.e., one with $l^T x = V$), solvers can fail to solve the
     problem.

   </details>
</div>

You'll need one more bit of PICOS syntax:

| on paper | in picos |
| --- | --- |
| $$\begin{bmatrix}A & B\end{bmatrix}$$ | `A & B` |
| $$\begin{bmatrix}A \\ B\end{bmatrix}$$ | `A // B` |
| $$\begin{bmatrix}A & B \\ C & D\end{bmatrix}$$ | `pc.block([[A, B], [C, D]])` |

The following constants are defined at this point:

| identifier | meaning |
| --- | --- |
| `crane.n` | number of nodes $n$ |
| `crane.m` | number of bars $m$ |
| `crane.d` | dimensionality $d$ |
| `l` | bar lengths $l$ as a NumPy $m$-vector |
| `V` | maximum total bar volume $V$ |

In [ ]:
# Some size estimates that can help you avoid numeric problems.
# As an estimate of x at optimality, we use a uniform vector x with <l, x> = V.
norm_f = np.linalg.norm(F)
est_opt_x = np.full(crane.m, V / L)
est_opt_K = crane.stiffness_matrix(est_opt_x)
est_norm_x = np.linalg.norm(est_opt_x)
est_norm_K = np.linalg.norm(est_opt_K, 2)

print(
    f"Estimated norms at optimality:\n"
    f"f:     {norm_f:.1e}\n"
    f"x:     {est_norm_x:.1e}\n"
    f"K(x):  {est_norm_K:.1e}\n"
)

# TODO: Define and solve the problem.

# Show the solution.
crane.plot(x.np, F)
crane.print_stats(x.np, F)

Your crane should now have a lower compliance with the specified force, leading
to a significantly reduced displacement at the tip!

### Bonus problem: Robust truss design

In classical conic optimization, we optimize problems of the form

$$
\begin{align*}
    \text{minimize} ~&~ f(x) \\
    \text{subject to} ~&~ g(x) \preceq_K 0,
\end{align*}
$$

where $x$ is a decision variable and where the functions $f$ and $g$ encode the
*data* of the problem. The problems we've solved so far are examples of this.
With this approach, it can happen that we find optimal solutions that turn into
really bad solutions if the data changes just a little. Unfortunately, truss
design is no exception to this ([Ben-tal and Nemirovski, 1997](#references))!

In *robust optimization*, one considers **uncertainty in the data** and solves
min-max problems of the form

$$
\begin{align*}
    \text{minimize} ~&~ \sup_{\theta \in \Theta} f(x, \theta) \\
    \text{subject to}
    ~&~ g(x, \theta) \preceq_K 0 \quad \forall~\theta \in \Theta,
\end{align*}
$$

where $\Theta$ is called a *perturbation set*. The *perturbation parameter*
$\theta \in \Theta$ is used to describe possible outcomes of the uncertain data.
In truss design, for instance, one could consider small perturbations to the
force vector to model the effect of wind.

Robust optimization is a pessimistic approach to deal with the uncertainty: one
optimizes with respect to the worst possible outcome and requires constraints to
hold in any case. (For a different approach, see [stochastic
programming](https://en.wikipedia.org/wiki/Stochastic_programming).)

Let's next design a crane that withstands a small force acting on any subset of
its joints and in arbitrary directions:

$$
\begin{align*}
    \text{minimize} ~&~
    \sup_{\lVert \theta \rVert_2 = 1} \theta^T {K(x)}^{-1} \theta \tag{3} \\
    \text{subject to}
    ~&~ l^T x \leq V, \\
    ~&~ x \geq 0, \\
    \text{where}
    ~&~ x \in \mathbb{R}^m.\\
\end{align*}
$$

---

<div class="alert alert-info">

   **Bonus task 3.5:**

   1. Have a close look at the objective of problem (3). Have you seen such an
      expression before?
   2. Can you repose the objective as a maximization objective?

   <details>
   <summary style='display: list-item'>Hint 1</summary>

   - Recall from the introductory lecture that
   
     $$
        \sup_{\lVert \theta \rVert_2 = 1} \theta^T {K(x)}^{-1} \theta
        = \lambda_{\max}\left( {K(x)}^{-1} \right).
     $$

   </details>

   <details>
   <summary style='display: list-item'>Hint 2</summary>

   - Every real symmetric matrix $A$ can be diagonalized as $A = Q^T D Q$ where
     $Q$ is an orthogonal and $D$ a diagonal matrix whose diagonal elements are
     the eigenvalues of $A$. What are thus the eigenvalues of $A^{-1}$?

   </details>
</div>

---

<div class="alert alert-info">

   **Bonus task 3.6:** Implement the maximization variant of problem (3).

   It's up to you whether you want to find the SDP representation by hand or let
   PICOS do it:

   | on paper | in picos |
   | --- | --- |
   | $$\lambda_{\min}(A)$$ | `pc.lambda_min(A)` |
   | $$\lambda_{\max}(A)$$ | `pc.lambda_max(A)` |

   Note that you can re-use your constants and variables from above.

</div>

In [ ]:
# TODO: Implement a suited reformulation of problem (3).

# Plot the design using the same force as before.
crane.plot(x.np, F)
crane.print_stats(x.np, F)

This construction is maybe a good idea if you have to support every possible
joint and don't know what kind of forces to expect. Of course, this is not
really a realistic scenario for a crane. Can we do better?

Let's see if we can consider also a force that acts only on a subset of the
nodes, is uncertain, but can be stronger in certain directions. In other words,
let's consider a linear function $A \theta$ for a fixed $A \in \mathbb{R}^{n
\times \ell}$:

$$
\begin{align*}
    \text{minimize} ~&~
    \sup_{\theta \in \mathbb{R}^\ell \colon \lVert A \theta \rVert_2 = 1}
        (A \theta)^T {K(x)}^{-1} (A \theta) \tag{4} \\
    \text{subject to}
    ~&~ l^T x \leq V, \\
    ~&~ x \geq 0, \\
    \text{where}
    ~&~ x \in \mathbb{R}^m.\\
\end{align*}
$$

---

<div class="alert alert-info">

   **Bonus task 3.7:** Repose problem (4) as an SDP.

   <details>
   <summary style='display: list-item'>Hint</summary>

   - You'll need most of what you've learned so far, including the Schur
     complement!
   - First, use an epigraph reformulation and
     $$
        \sup_{\theta \in \mathbb{R}^\ell \colon \lVert A \theta \rVert_2 = 1}
            f(x, \theta)
        =
        \sup_{\theta \in \mathbb{R}^\ell \colon A \theta \neq 0}
            \frac{f(x, \theta)}{{(A \theta)}^T (A \theta)}.
     $$

   </details>
</div>

---

<div class="alert alert-info">

   **Bonus task 3.8:** Implement problem (4) for the given $A$.

   <details>
   <summary style='display: list-item'>Hack to improve numeric stability</summary>

   - CVXOPT does not appreciate this problem and MOSEK takes its time. You can
     work around both by scaling the volume $V$ by a factor, say $100$, and
     dividing the solution $x$ by the same amount.

   </details>
</div>

In [ ]:
# Let {Aθ | ‖θ‖ = 1} be a 2D ellipsoid centered at the crane's tip, whose major
# axis is aligned vertically and is α times as long as the minor axis.
alpha = 3
tip_up_col_vec = tip_up.ravel()[crane.mask][:, np.newaxis]
tip_rt_col_vec = tip_rt.ravel()[crane.mask][:, np.newaxis]
A = pc.Constant("A", np.hstack([alpha * tip_up_col_vec, tip_rt_col_vec]))
print(repr(A))

# TODO: Solve problem (4) and plot the design.


If you compare this truss to the solution of task 3.4, it should look more
robust with respect to horizontal forces!

### Additional resources

- Section 2 of Aranda and Bellido ([2016](#references)) explains the
  construction of the stiffness matrix.
- UC Boulder has a nice [writeup on truss
  design](https://www.teachengineering.org/lessons/view/ind-2472-analysis-forces-truss-bridge-lesson)
  aimed at engineering instructors.

### Acknowledgements

This exercise is based on a notebook by Guillaume Sagnol.

## References

- Ernesto Aranda and José C. Bellido, 2016. Introduction to truss structures
  optimization with Python.
  https://josecarlosbellido.files.wordpress.com/2016/04/aranda-bellido-optruss.pdf
- Aharon Ben-Tal and Arkadi Nemirovski, 1997. Robust truss topology design via
  semidefinite programming. *SIAM Journal on Optimization* 7, 4. 991-1016.
  https://doi.org/10.1137/S1052623495291951